# Tratamiento de datos

In [2]:
import numpy as np
import pandas
import re
from data import DATA
import matplotlib.pyplot as plt

## 1. Definición de constantes

In [3]:
CORPUS_DIR = "../static/corpus"
CORPUS_PRE = "Twitter_Colombia_"
CORPUS_EXT = "tsv"
CORPUS_COL = "tweet"

## 2. Funciones de tratamiento de archivos

In [4]:
def CorpusToDataframe(city, folder, prefix, ext, corpus_col):
    '''
    Lee archivos tabulados de corpus de tweets con la siguente estructura de columnas:
    
    user_id: Identificación numérica del usuario.
    YYYY-MM-DD: Fecha del tweet
    tweet: el tweet.
    replies: Número de respuestas
    retweets: Número de retweets
    likes: Número de likes
    
    Solo las columnas "user_id" y "tweet" son requeridas.
    
    Parameters
    ----------
    city: str
        Nombre de la ciudad a procesar.
    folder: 
        Carpeta donde se encuentra el archivo.
    prefix: str
        Porción  del nombre del archivo que aparece antes del nombre de 
        la ciudad, en este caso "Twitter_Colombia_"
    ext:   str
        Extensión del archivo
    corpus_col:  str
    
    Returns
    -------
    users: int
        Número de usuarios que realizaron los twits
    tweets_per_user: Pandas.DataFrame 
        Arreglo de datos de panda con el user_id y el conteo de tweets por usuario
    corpus: Pandas.DataFrame
         Arreglo de datos de panda con el corpus y la estructura de columnas ["user_id",["tweet"]
    
    '''
    ORIGIN = '{}/{}{}.{}'.format(folder,prefix,city,ext)
    corpus = pandas.read_csv(ORIGIN, sep="\t", encoding='utf-8', usecols=["user_id","tweet"])
    users = corpus["user_id"].max()
    tweets_per_user = corpus.groupby("user_id").count()
    return (users, tweets_per_user, corpus)


In [1]:
def  RawCorpusToBagOfWords(corpus):
    '''
    Toma un dataframe de corpus y lo convierte en una bolsa de palabras con usuarios asociados
    
    Parameters
    ----------
    corpus: Pandas.DataFrame
        Arreglo de datos del corpus con estructura de columnas ["user_id"],["tweet"]

    Returns
    -------
    Pandas.DataFrame
    '''
    
    corpus_len = len(corpus.index)
    for index, row in corpus.iterrows():
        print('\rProcesando tweet {} de {}'.format(index,corpus_len), end='\t\t\t\t')
        tweet_words=pandas.DataFrame(row).transpose()
        user_id = tweet_words.user_id
        tweet_words=tweet_words.tweet.str.split(" ", expand=True).transpose()
        if index == 0:
            bagofwords=tweet_words
        else:
            bagofwords=bagofwords.append(tweet_words)
        
    bagofwords.columns = ['words']
    bagofwords = bagofwords[['words']]
    return(bagofwords)


In [6]:
users, tweets_per_user, corpus = CorpusToDataframe('Abrego', CORPUS_DIR,  CORPUS_PRE, CORPUS_EXT, CORPUS_COL)
    
RawCorpusToBagOfWords(corpus)



Procesando tweet 1629 de 3711				

KeyboardInterrupt: 

In [7]:
users, tweets_per_user, corpus = CorpusToDataframe('Bogota', CORPUS_DIR,  CORPUS_PRE, CORPUS_EXT, CORPUS_COL)
RawCorpusToBagOfWords(corpus)


Procesando tweet 1733 de 2990417				

KeyboardInterrupt: 

In [5]:
def clean_links(bagofwords):
    '''
    Toma una bolsa de palabras y le quita los enlaces.
    
    Parameters
    ----------
    corpus: Pandas.DataFrame
        Arreglo de datos del corpus con estructura de columnas ["user_id",["tweet"]

    Returns
    -------
    Pandas.DataFrame
    '''
    regex_pat = re.compile(r'^https?:\/\/.*[\r\n]*',flags=re.IGNORECASE)
    bagofwords['link'] = bagofwords.tweet.str.contains(regex_pat)
    bagofwords = bagofwords.loc[bagofwords['link'] == False]
    #bagofwords = bagofwords.filter(like=True)
    return(bagofwords)

In [7]:
users, tweets_per_user, corpus = CorpusToDataframe('Abrego', CORPUS_DIR,  CORPUS_PRE, CORPUS_EXT, CORPUS_COL)
print(corpus.info())
bagofwords=RawCorpusToBagOfWords(corpus)
del corpus
bagofwords
#clean_links(bagofwords)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 2 columns):
user_id    3711 non-null int64
tweet      3711 non-null object
dtypes: int64(1), object(1)
memory usage: 58.1+ KB
None


,user_id,tweet
0,1,@
1,1,Fin
2,1,@
3,2,@classicshirtsAntonio
4,3,Amor
5,4,Amor
6,5,Replying
7,6,Pegándole
8,7,@
9,8,Esta
